# imports and setup

In [ ]:
# Jupyter
%matplotlib inline

# Env + basics
import os
import pandas as pd

# Project modules
from pipeline.engine import run_pipeline
from pipeline.reporting import show_run_report
from pipeline.review import ReviewConfig

# Load data (ohne parse_dates)
file_path = r"path_to_file"
df = pd.read_csv(file_path, sep=",", on_bad_lines="skip", engine="python").reset_index(drop=True)

print(len(df))
df.head(12)

## run example openai

In [ ]:
# API key + model (used by project/config.py)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "sk-...")

prompt = "Show me the distribution of ta rel, only finished parts"

state, df_out, results = await run_pipeline(
    prompt=prompt,
    df=df,
    review_config=ReviewConfig(
        enabled=True,
        #after_steps={"family","type","filters","columns","prepare","aggregate","viz","analyze","verify","finalize"},
        after_steps={},
        show_step_inputs=True,
    )
)

show_run_report(
    results,
    show_head_df=df_out,
    show_step_inputs=False,   # <- nicht doppelt
    verbose_steps=False,      # <- keine Step-Dumps am Ende
    render_final=True,        # <- Plots + Final text
    final_head_rows=5,
)

## run example openrouter

In [ ]:
os.environ["OPENROUTER_API_KEY"] = "sk-..."

In [ ]:
from agents.factory import AgentFactory
from agents.llm_openrouter import OpenRouterJSONLLM

openrouter_llm = OpenRouterJSONLLM(
    model="deepseek/deepseek-r1-0528:free",
    api_key=os.environ["OPENROUTER_API_KEY"],
    app_url="http://localhost",            
    app_name="data_agent_deeping",         
    max_output_tokens=4096,
)

agents = AgentFactory(llm=openrouter_llm)

In [ ]:
prompt = "Show me the development of average cycle time per month by ta status, only finished parts"

state, df_out, results = await run_pipeline(
    prompt=prompt,
    df=df,
    agents=agents,
    review_config=ReviewConfig(
        enabled=True,
        #after_steps={"family","type","filters","columns","prepare","aggregate","viz","analyze","verify","finalize"},
        after_steps={},
        show_step_inputs=True,
    )
)

show_run_report(
    results,
    show_head_df=df_out,
    show_step_inputs=False,
    verbose_steps=False,
    render_final=True,
    final_head_rows=5,
)